In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.distributions.mixture_rvs import mixture_rvs

In [2]:
# import the csv file
df = pd.read_csv('./prepared_data/2020_ffl_df.csv')

# clean up the kde_icdf column
for i in range(0,len(df)):
    df['kde_icdf'][i] = df['kde_icdf'][i].replace('\n',' ').replace('[','').replace(']','')
    df['kde_icdf'][i] = [float(x) for x in df['kde_icdf'][i].split()]
    
# more data cleanup
df['POS']=df['POS'].str.strip()

# add drafted column to track drafted players
df['drafted']=0

In [3]:
def compare_best_players(positions):
    # identify best players per position
    best_players = []
    j = 0
    for j in range(0, len(positions)):
        k = 0
        for k in range(0, len(df)):
            if df['drafted'][k] == 1:
                k += 1
            if df['drafted'][k] == 0:
                if df['POS'][k] == positions[j]:
                    if len(best_players) > j:
                        if df['pts'][k] > best_players[j][2]:
                            best_players[j] = [df['POS'][k], df['Name'][k], df['pts'][k], df['Draft Position'][k]]
                            k += 1
                        if df['pts'][k] <= best_players[j][2]:
                            k += 1
                    if len(best_players)<=j:
                        best_players.append([df['POS'][k], df['Name'][k], df['pts'][k],df['Draft Position'][k]])
                        k += 1

    return best_players

In [ ]:
def compare_best_players_next_round(positions, next_available_pick):
    # identify best choice in next round at each position                
    best_players_next_round = []
    j = 0
    for j in range(0, len(positions)):
        k = 0
        for k in range(0, len(df)):
            if df['drafted'][k] == 1:
                k += 1
            if df['drafted'][k] == 0:
                if df['POS'][k] == positions[j]:
                    if len(best_players_next_round) > j:
                        if df['Draft Position'][k] < next_available_pick:
                            k += 1
                        else:
                            if df['pts'][k] > best_players_next_round[j][2]:
                                best_players_next_round[j] = [df['POS'][k], df['Name'][k], df['pts'][k], df['Draft Position'][k]]
                                k += 1
                            if df['pts'][k] <= best_players_next_round[j][2]:
                                k += 1
                    if len(best_players_next_round) <= j:
                        if df['Draft Position'][k] < next_available_pick:
                            k += 1
                        else:
                            best_players_next_round.append([df['POS'][k], df['Name'][k], df['pts'][k], df['Draft Position'][k]])
                            k += 1

    return best_players_next_round

In [4]:
def compare_second_best_players(positions, best_players):
    # loop to collect second-best choices for each round
    second_best_players = []
    j = 0
    for j in range(0,len(positions)):
        k = 0
        for k in range(0,len(df)):
            if df['drafted'][k] == 1:
                k += 1
            if df['drafted'][k] == 0:
                if df['POS'][k] == positions[j]:
                    if len(second_best_players) > j:
                        if df['Name'][k] == best_players[j][1]:
                            k += 1
                        else:
                            if df['pts'][k] > second_best_players[j][2]:
                                second_best_players[j]=[df['POS'][k], df['Name'][k], df['pts'][k], df['Draft Position'][k]]
                                k += 1
                            if df['pts'][k] <= second_best_players[j][2]:
                                k += 1
                    if len(second_best_players) <= j:
                        if df['Name'][k] == best_players[j][1]:
                            k += 1
                        else:
                            second_best_players.append([df['POS'][k], df['Name'][k], df['pts'][k], df['Draft Position'][k]])
                            k += 1 

In [8]:
def DAsHa_prgm(turns_to_next_pick):
    # to find current draft number
    current_draft_number = 0
    i = 0
    while current_draft_number == 0:
        if df['drafted'][i] == 1:
            i += 1
        else:
            current_draft_number = df['Draft Position'][i]

    # threshold for next available picks
    next_available_pick = current_draft_number + turns_to_next_pick

    # List of possible positions to draft for
    positions = ['QB','RB','WR','TE','K','DEF']

    # identify best choices at each position
    best_players = compare_best_players(positions)

    # identify best choice in next round at each position                
    best_players_next_round = compare_best_players_next_round(positions, next_available_pick)

    # collect the second-best choices for each round
    second_best_players = compare_second_best_players(positions, best_players)

    # make marginal comparison and append to best_players list
    for j in range(0,len(positions)):
        marg = best_players[j][2] - best_players_next_round[j][2]
        best_players[j].append(marg)

    # print recommendation menu
    for i in range(0,len(positions)):
        print(f"{best_players[i][0]:{5}} - {best_players[i][1]:{20}} - {round(best_players[i][2],2):{6}}, margin: {round(best_players[i][4],2)}")

    # find top two second-best
    top_seconds = [second_best_players[0][2], second_best_players[1][2], second_best_players[2][2], second_best_players[3][2], second_best_players[4][2], second_best_players[5][2],]

    # finding index values (based off Yahoo! draft position) to call the boxplots
    DEF = best_players[0][3] - 1
    K = best_players[1][3] - 1
    TE = best_players[2][3] - 1
    WR = best_players[3][3] - 1
    RB = best_players[4][3] - 1
    QB = best_players[5][3] - 1

    # boxplots of recs
    box_plot_data=[df['kde_icdf'][QB], df['kde_icdf'][RB], df['kde_icdf'][WR], df['kde_icdf'][TE], df['kde_icdf'][K], df['kde_icdf'][DEF]]
    plt.boxplot(box_plot_data, vert=False, labels=[df['Name'][QB], df['Name'][RB], df['Name'][WR], df['Name'][TE], df['Name'][K], df['Name'][DEF] ])
    plt.show()

    # find top two second-best
    top_seconds = [second_best_players[0][2], second_best_players[1][2], second_best_players[2][2], second_best_players[3][2],second_best_players[4][2],second_best_players[5][2],]
    first_comparison = top_seconds.index(max(top_seconds))
    top_seconds[first_comparison] = 0
    second_comparison = top_seconds.index(max(top_seconds))

    # boxplots of second best
    first_best_first = best_players[first_comparison][3] - 1
    second_best_first = second_best_players[first_comparison][3] - 1
    first_best_second = best_players[second_comparison][3] - 1
    second_best_second = second_best_players[second_comparison][3] - 1

    box_plot_data=[df['kde_icdf'][second_best_first], df['kde_icdf'][first_best_first]]
    plt.boxplot(box_plot_data, vert=False, labels=[df['Name'][second_best_first], df['Name'][first_best_first]])
    plt.show()

    box_plot_data=[df['kde_icdf'][], df['kde_icdf'][first_best_second]]
    plt.boxplot(box_plot_data, vert=False, labels=[df['Name'][], df['Name'][first_best_second]])
    plt.show()

In [9]:
### This is the user-interace: enter the picked players until it's your turn. Then, enter "DAsHA_prgm(# of rounds to your next pick)" in the cell below.


In [6]:
your_turn = False
while your_turn == False:
    new_draft = input('Who was the last pick?')
    if new_draft != '':
        temp_index = df['Name'].str.find(str(new_draft.strip()))
        temp_no = temp_index.values.argmax()
        df['drafted'][temp_no] = 1
        new_draft = ''
    your_turn_input = input('Is it your turn?')
    no_set = {'no', 'n', 'N', 'No'}   
    if your_turn_input in no_set: 
        your_turn = False
    else:
        your_turn = True


In [10]:
DAsHa_prgm(14)

UnboundLocalError: local variable 'current_draft_number' referenced before assignment